In [89]:
# Медузa: 2014.10.09 - 2019.08.06 (71 806)
# Lenta:     1999.08.30 -  2018.12.15 (800 975)
# Gazeta:   2008.11.21 -  (865 847)

### Медуза

In [1]:
pip install psycopg2 fasttext pandas sqlalchemy corus

  Using cached psycopg2-2.9.5-cp310-cp310-linux_x86_64.whl
  Using cached fasttext-0.9.2-cp310-cp310-linux_x86_64.whl
  Using cached pandas-1.5.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.1 MB)
  Using cached SQLAlchemy-1.4.45-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.6 MB)
  Using cached corus-0.9.0-py3-none-any.whl (83 kB)
  Using cached pybind11-2.10.1-py3-none-any.whl (216 kB)
  Using cached numpy-1.23.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.1 MB)
  Using cached pytz-2022.6-py2.py3-none-any.whl (498 kB)
  Using cached greenlet-2.0.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (539 kB)
You should consider upgrading via the '/home/maxlethal/Yandex.Disk/IT/Projects/archiveSMI/venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import fasttext
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

In [2]:
arh_engine = create_engine(f'postgresql+psycopg2://{db_user}:{db_pass}@{db_host}/{db_name}')

In [3]:
model_fasttext = fasttext.load_model("models//ru_cat_v5.ftz")

In [12]:
from corus import load_ods_meduza

path = 'datasets/meduza.csv.gz'
records = load_ods_meduza(path)

In [11]:
current_news = next(records)

In [8]:
current_news

NewsRecord(
    timestamp=datetime.datetime(2019, 8, 6, 14, 15),
    url='https://meduza.io/feature/2019/08/06/matros-razgovarival-v-kazarme-po-videosvyazi',
    edition=None,
    topics=None,
    authors=None,
    title='Матрос разговаривал в\xa0казарме по\xa0видеосвязи',
    text='С\xa0марта 2019 года российским военным запрещено пользоваться смартфонами и\xa0публиковать в\xa0интернете фотографии со\xa0службы. Соответствующие поправки к\xa0закону «О\xa0статусе военнослужащих» правительство подготовило в\xa0сентябре 2018 года, а\xa0Госдума и\xa0Совет Федерации одобрили их\xa0в\xa0феврале 2019-го. В\xa0марте закон подписал президент России Владимир Путин, и\xa017\xa0марта онnвступилnв\xa0силу. Российских военных сразу\xa0же начали наказывать за\xa0найденные у\xa0них смартфоны и\xa0фото в\xa0социальных сетях: обычно им\xa0дают от\xa0пяти до\xa015 суток ареста, выяснила «Медуза», изучив решения военных судов.nВ\xa0пояснительной записке к\xa0поправкам правительствоnсвязывалоnих\xa0необход

In [6]:
index = current_news.url
date = current_news.timestamp
title = current_news.title.replace("\xa0", ' ').replace("\n", ' ')
text = current_news.text.replace("\xa0", ' ').replace("\n", ' ')
tags = current_news.topics
category = model_fasttext.predict(title)[0][0].split('__')[-1]
agency = 'meduzalive'

In [7]:
dict_news = {'url': index, 'date': date, 'title': title, 'news': text, 'tags': tags, 'category': category, 'agency': agency}

In [16]:
dict_news

{'url': 'https://meduza.io/news/2019/08/06/umerla-feshn-direktor-tsuma-alla-verber',
 'date': datetime.datetime(2019, 8, 6, 14, 20),
 'title': 'Умерла фэшн-директор ЦУМа Алла Вербер',
 'news': 'Умерла фэшн-директор московского ЦУМа, вице-президент ювелирной компании Mercury Алла Вербер. Эту информацию изданию Hello подтвердили в пресс-службе ЦУМа.nДруг Вербер, журналист Олег Шаранnнаписалnв фейсбуке, что Вербер умерла на курорте в Италии; причиной смерти стал инфаркт. Издание Super.runутверждаетn, что его спровоцировал анафилактический шок после ужина с блюдами из морепродуктов.nАлле Вербер был 61 год.nАлла Вербер родилась в Ленинграде. В 18 лет вместе с семьей уехала из СССР. Жила в Италии, США и Канаде, там же начала работу в фэшн-бизнесе. В 1989 году Вербер вернулась в СССР, а последующие годы стала одной из первых, кто привез в Россию западные модные бренды. В 1995 году Вербер начала сотрудничать с компанией Mercury, которая сейчасnявляетсяnединственным владельцем московского ЦУМа.

In [22]:
test_df = pd.DataFrame.from_dict(dict_news, orient='index').T

ValueError: If using all scalar values, you must pass an index

In [19]:
test_df

,url,date,title,news,tags,category,agency
0,https://meduza.io/news/2019/08/06/umerla-feshn...,2019-08-06 14:20:00,Умерла фэшн-директор ЦУМа Алла Вербер,"Умерла фэшн-директор московского ЦУМа, вице-пр...",None,society,meduzalive


In [56]:
date_df = pd.DataFrame.from_dict(dict_news, orient='index').T.set_index('index')

In [67]:
date_df.to_sql(name='news', con=engine, if_exists='append', index=True)

1

In [21]:
while True:
    current_news = next(records)
    index = current_news.url
    date = current_news.timestamp
    if current_news.title:
        title = current_news.title.replace("\xa0", ' ').replace("\n", ' ')
    else:
        title = current_news.title
    if current_news.text:
        text = current_news.text.replace("\xa0", ' ').replace("\n", ' ')
    else:
        text = current_news.text
    tags = current_news.topics
    if current_news.title:
        category = model_fasttext.predict(title)[0][0].split('__')[-1]
    else:
        category = 'NaN'
    agency = 'meduzalive'
    dict_news = {'url': index, 'date': date, 'title': title, 'news': text, 'tags': tags, 'category': category, 'agency': agency}
    date_df = pd.DataFrame.from_dict(dict_news, orient='index').T
    date_df.to_sql(name='news', con=arh_engine, if_exists='append', index=False)

StopIteration: 

### Лента

In [23]:
from corus import load_lenta2

path = 'datasets/lenta-ru-news.csv.bz2'
records = load_lenta2(path)

In [ ]:
while True:
    current_news = next(records)
    index = current_news.url
    date = current_news.date
    if current_news.title:
        title = current_news.title.replace("\xa0", ' ').replace("\n", ' ')
    else:
        title = current_news.title
    if current_news.text:
        text = current_news.text.replace("\xa0", ' ').replace("\n", ' ')
    else:
        text = current_news.text
    tags = current_news.tags
    category = current_news.topic
    agency = 'lenta'
    dict_news = {'url': index, 'date': date, 'title': title, 'news': text, 'tags': tags, 'category': category, 'agency': agency}
    date_df = pd.DataFrame.from_dict(dict_news, orient='index').T
    date_df.to_sql(name='news', con=arh_engine, if_exists='append', index=False)

### Gazeta

In [6]:
from corus import load_ods_gazeta

path = 'Datasets/gazeta.csv.gz'
records = load_ods_gazeta(path)

In [7]:
while True:
    current_news = next(records)
    index = current_news.url
    date = current_news.timestamp
    if current_news.title:
        title = current_news.title.replace("\xa0", ' ').replace("\n", ' ')
    else:
        title = current_news.title
    if current_news.text:
        text = current_news.text.replace("\xa0", ' ').replace("\n", ' ')
    else:
        text = current_news.text
    tags = current_news.topics
    if current_news.title:
        category = model_fasttext.predict(title)[0][0].split('__')[-1]
    else:
        category = 'NaN'
    agency = 'gazeta'
    dict_news = {'url': index, 'date': date, 'title': title, 'news': text, 'tags': tags, 'category': category, 'agency': agency}
    date_df = pd.DataFrame.from_dict(dict_news, orient='index').T
    date_df.to_sql(name='news', con=arh_engine, if_exists='append', index=False)

StopIteration: 

### NPlusOne

In [12]:
from corus import load_taiga_nplus1_metas, load_taiga_nplus1

path = 'Datasets/NPlus1.tar.gz'
metas = load_taiga_nplus1_metas(path, offset=0, count=1)
records = load_taiga_nplus1(path, metas, offset=0, count=7696)

In [137]:
current_news = next(records)

In [138]:
current_news

TaigaRecord(
    id='20150302chaotic',
    meta=Meta(
        id='20150302chaotic',
        timestamp=datetime.datetime(2015, 3, 2, 17, 53),
        tags=None,
        themes=None,
        rubric='Наука',
        genre=None,
        topic=None,
        author=Author(
            name='AZ ZAZ',
            readers=None,
            texts=None,
            profession=None,
            about=None,
            url=None
        ),
        lang=None,
        title='Физики поделили экономики на «ламинарные» и «хаотические»',
        url='https://nplus1.ru/news/2015/03/02/chaotic'
    ),
    text='\nУченые из Университета Сапиенца в Риме предложили метод классификации стран в соответствии с типом их экономики. Метод позволяет сказать, насколько поведение страны поддается прогнозу и соответствует ли оно «ламинарному» или «хаотическому» типу развития. Работа опубликована\xa0в журнале PLoS ONE, кратко о ней можнопрочитать\xa0на сайте Nature. Главной задачей исследования был поиск метрики, которая

In [13]:
while True:
    current_news = next(records)
    index = current_news.meta.url
    date = current_news.meta.timestamp
    if current_news.meta.title:
        title = current_news.meta.title.replace("\xa0", ' ').replace("\n", ' ')
    else:
        title = current_news.meta.title
    if current_news.text:
        text = current_news.text.replace("\xa0", ' ').replace("\n", ' ')
    else:
        text = current_news.text
    tags = current_news.meta.tags
    category = 'science'
    agency = 'nplusone'
    dict_news = {'url': index, 'date': date, 'title': title, 'news': text, 'tags': tags, 'category': category, 'agency': agency}
    date_df = pd.DataFrame.from_dict(dict_news, orient='index').T
    date_df.to_sql(name='news', con=arh_engine, if_exists='append', index=False)

StopIteration: 